In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.datasets import MNIST
import matplotlib.pyplot as plt
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28*28, 64)
        self.fc2 = torch.nn.Linear(64, 64)
        self.fc3 = torch.nn.Linear(64, 64)
        self.fc4 = torch.nn.Linear(64, 10)
    def forward(self, x):
        x=torch.nn.functional.relu(self.fc1(x))
        x=torch.nn.functional.relu(self.fc2(x))
        x=tor


,dta,ref1,ref2,ref3,ref4,ref5,ref6,ref7,par,parc,sm,ta,cos,sif,count,ndvi,evi,nirv,ndwi,vpd
0,-5.091370,1195.5,100.0,0.0,32767.0,1253.0,869.0,1141.5,118.015396,159.198853,0.137777,-3.390640,0.342788,0.000000,1.0,-0.845619,-0.376512,-84.561945,-0.852180,0.057268
1,-5.091370,943.0,100.0,0.0,32767.0,1013.5,702.0,918.5,118.015396,159.180435,0.137777,-3.390640,0.342788,0.000000,1.0,-0.808245,-0.365949,-80.824545,-0.820386,0.057268
2,-5.073395,759.5,100.0,17.5,32767.0,709.5,646.5,777.0,118.058243,159.180435,0.138349,-3.383881,0.342788,0.000000,1.0,-0.767307,-0.364224,-76.730657,-0.752934,0.056937
3,-5.073395,1062.5,100.0,33.0,32767.0,1046.0,934.0,1107.5,118.058243,159.180435,0.138349,-3.383881,0.342788,0.000000,1.0,-0.827957,-0.386329,-82.795699,-0.825480,0.056937
4,-5.054794,1219.5,99.0,33.0,32767.0,1162.0,1068.5,1236.5,118.269005,159.180435,0.138899,-3.377136,0.342788,0.000000,1.0,-0.849829,-0.390718,-84.133106,-0.842982,0.056585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4052080,-3.767426,2088.5,42.0,0.0,32767.0,2202.0,2247.0,2021.0,81.686737,102.889961,0.382757,-0.621918,0.378521,-0.187063,1.0,-0.960573,-0.406891,-40.344051,-0.962567,0.121424
4052081,-3.496246,110.0,43.0,0.0,32767.0,93.0,298.0,186.0,79.792152,102.623779,0.379867,-0.113968,0.365522,-0.072707,2.0,-0.437908,-0.237926,-18.830065,-0.367647,0.133970
4052082,-3.268600,617.0,61.0,0.0,32767.0,1010.0,624.0,603.0,79.865860,102.623779,0.380894,0.288177,0.377205,0.088679,3.0,-0.820059,-0.369288,-50.023599,-0.886088,0.143834
4052083,-2.239960,3840.0,4.0,0.0,32767.0,4840.0,3384.0,3900.0,78.780594,98.206497,0.395493,2.387543,0.370113,0.014395,1.0,-0.997919,-0.416142,-3.991675,-0.998348,0.207338
